In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import h5py
%matplotlib inline

mat_names = glob.glob('./project_datasets/*.mat')
# each test subject got a different file - 9 test subjects
print(mat_names)

In [ ]:
matfile = h5py.File(mat_names[0], 'r')
print(matfile.keys()) #image and type

In [ ]:
image_mat = matfile['image']
image_shape = image_mat.shape # 288 (48x6) trials across 25 electrodes for 1000 time points (250Hz*4s)
print image_shape

In [ ]:
type_mat = matfile['type']
type_shape = type_mat.shape
print type_shape
# plt.plot(type_mat[0,:288]) # gets the significant values of types
# all the 0's occur after 288, and are meaningless I think
# so the image_mat, which has shape (288, 25, 1000) should correspond
# to the first 288 entries of type_mat, so
# for a single subject, training data should be image_mat, with 288 samples, each sample has shape (25, 1000)
# and our target label matrix should be type_mat[:288] (or 287?)

In [ ]:
nans = np.sum(np.isnan(image_mat[:,:]))
print(nans) #No NaN in the data
print len(image_mat[0:,:])
count = 0
# for i in range(len(image_mat[0:,:])):
#  if np.sum(np.isnan(image_mat[i:,:])):
#         pass

In [ ]:
type_set = list(set(type_mat[0,:]))
print(type_set) 

In [ ]:
for i in range(image_shape[1]):
    plt.subplot(image_shape[1]**0.5,image_shape[1]**0.5,i+1)
    plt.plot(matfile['image'][0,i,:])
    plt.title("Electrode " + str(i))
    plt.rcParams["figure.figsize"] = [20,20]

In [ ]:
plt.plot(matfile['type'][0,:image_shape[0]])

In [ ]:
EEG_channels = 22 #from project guidelines
test_count = 50 #from project guideline, 238 for train-validation and 50 for test
validation_count = 38 # 38 points in validation set and remaining 200 points in test set

In [ ]:
#setting seed
np.random.seed(seed=1337)
test_picked = np.random.choice(image_shape[0], test_count, replace=False)
train_val_picked = np.setdiff1d(np.arange(image_shape[0]), test_picked)
val_picked = train_val_picked[:validation_count]
train_picked = train_val_picked[validation_count:]

In [ ]:
trainval_data_X = []
training_data_X = []
validation_data_X = []
test_data_X = []

trainval_data_Y = []
training_data_Y = []
validation_data_Y = []
test_data_Y = []

for i in range(len(mat_names)):
    matfile = h5py.File(mat_names[i], 'r')
    
    trainval_data_X.append(matfile['image'][sorted(train_val_picked),:EEG_channels,:]) #(238, 22, 1000) x 9
    training_data_X.append(matfile['image'][sorted(train_picked),:EEG_channels,:]) #(200, 22, 1000) x 9
    validation_data_X.append(matfile['image'][sorted(val_picked),:EEG_channels,:]) #(38, 22, 1000) x 9
    test_data_X.append(matfile['image'][sorted(test_picked),:EEG_channels,:]) #(50, 22, 1000) x 9
    
    trainval_data_Y.append(matfile['type'][0,sorted(train_val_picked)] - type_set[1]) #(238, ) x 9
    training_data_Y.append(matfile['type'][0,sorted(train_picked)] - type_set[1]) #(200, ) x 9
    validation_data_Y.append(matfile['type'][0,sorted(val_picked)] - type_set[1]) #(38, ) x 9
    test_data_Y.append(matfile['type'][0,sorted(test_picked)] - type_set[1]) #(50, ) x 9

In [ ]:
for i in range(len(mat_names)):
    assert(set(training_data_Y[i]) == set(validation_data_Y[i])) #check whether we have data for all labels 
    assert(set(training_data_Y[i]) == set(test_data_Y[i])) #check whether we have data for all labels 

In [ ]:
training_data_shape = training_data_X[0].shape
print(training_data_shape) #(200, 22, 1000) while test data shape is (50, 22, 1000) and validation data is (38, 22,1000)

In [ ]:
print(training_data_Y[0].shape)

In [ ]:
from functools import reduce

def remove_nan_rows_A(A, b):
    if (np.isnan(A).any() or np.isnan(b).any()):
        mask = ~np.isnan(np.sum(A,axis=(1,2))) & ~np.isnan(b[:])
        A = A[mask, :, :]
        b = b[mask]
        
    assert A.shape[0] == b.shape[0]
    return A, b


cnn_trainval_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), trainval_data_X) #(2142, 22, 1000) 
cnn_training_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), training_data_X) #(1800, 22, 1000)
cnn_validation_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), validation_data_X) #(342, 22, 1000)
cnn_test_data_X = reduce((lambda x, y: np.concatenate((x, y), axis=0)), test_data_X) #(450, 22, 1000)

cnn_trainval_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), trainval_data_Y) #(2142, )
cnn_training_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), training_data_Y) #(1800, )
cnn_validation_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), validation_data_Y) #(342, )
cnn_test_data_Y = reduce((lambda x, y: np.concatenate((x, y), axis=0)), test_data_Y) #(450,)


cnn_training_data_X, cnn_training_data_Y = remove_nan_rows_A(cnn_training_data_X, cnn_training_data_Y) #(1775,22,1000)
cnn_validation_data_X, cnn_validation_data_Y = remove_nan_rows_A(cnn_validation_data_X, cnn_validation_data_Y) #(340,22,1000)

cnn_test_data_X, cnn_test_data_Y = remove_nan_rows_A(cnn_test_data_X, cnn_test_data_Y) #(443,22,1000)

In [ ]:
print cnn_training_data_X.shape
print cnn_validation_data_X.shape
print cnn_test_data_X.shape

In [ ]:
# cnn_trainval_data_X = np.transpose(rnn_trainval_data_X, (0,2,1))
# mean_list = np.mean(cnn_trainval_data_X.reshape(-1, cnn_trainval_data_X.shape[-1]), axis=0)
# std_list = np.sqrt((np.var(cnn_trainval_data_X.reshape(-1, cnn_trainval_data_X.shape[-1]), axis=0)))
# cnn_training_data_X = (cnn_training_data_X - mean_list)/std_list
# cnn_validation_data_X = np.transpose(rnn_trainval_data_X, (0,2,1))

# print cnn_training_data_X.shape()
# print cnn_validation_data_X.shape()

In [ ]:
import sklearn
from sklearn import svm

In [ ]:
# svm_training_X = np.reshape(cnn_training_data_X, (1775, 22000))
# svm_training_Y = cnn_training_data_Y

In [ ]:
# clf = svm.SVC()
# clf.fit(svm_training_X, svm_training_Y)

In [ ]:
# svm_test_X = np.reshape(cnn_validation_data_X, (340, 22000))
# svm_test_Y = cnn_validation_data_Y
# predicted = clf.predict(svm_test_X)

# print(predicted)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import gc
import resource
print(torch.__version__)

In [ ]:
# helper function to get rss size, see stat(5) under statm. This is in pages...
def memory_usage():
    return int(open('/proc/self/statm').read().split()[1])


In [ ]:
num_epochs = 15
batch_size = 10
learning_rate = 3e-4

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.BatchNorm1d(22),
#             nn.Conv1d(22, 64, kernel_size=40), #32x1000
#             nn.BatchNorm1d(64), 
#             nn.ReLU(),
#             nn.MaxPool1d(2)) #32 x 500
#         self.layer2 = nn.Sequential(
#             nn.Conv1d(64, 32, kernel_size=40), #8x250
#             nn.BatchNorm1d(32),
#             nn.ReLU()) #8x250
#         self.layer3 = nn.Sequential(
#             nn.Conv1d(32, 16, kernel_size=40), #8x250
#             nn.BatchNorm1d(16),
#             nn.ReLU()) #8x250
#         self.layer4 = nn.Sequential(
#             nn.Conv1d(16, 8, kernel_size=40), #8x250
#             nn.BatchNorm1d(8),
#             nn.ReLU()) #8x250
#         self.fc1 = nn.Sequential(
#             nn.Linear(2904, 500), #100
#             nn.BatchNorm1d(500),
#             nn.ReLU())
# #         self.fc2 = nn.Sequential(
# #             nn.Linear(5000, 1000), #100
# #             nn.BatchNorm1d(1000),
# #             nn.ReLU())
# #         self.fc3 = nn.Sequential(
# #             nn.Linear(1000, 500), #100
# #             nn.BatchNorm1d(500),
# #             nn.ReLU())
# #         self.fc4 = nn.Sequential(
# #             nn.Linear(1000, 500), #100
# #             nn.BatchNorm1d(500),
# #             nn.ReLU())
#         self.fc5 = nn.Linear(500, 4)
        
#     #basic forward - go through two conv layers + fc layer
   
#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = self.layer4(out)
#         out = out.view(out.size(0), -1)
#         out = self.fc1(out)
# #         out = self.fc2(out)
# #         out = self.fc3(out)
# #         out = self.fc4(out)
#         out = self.fc5(out)
#         return out

#59% with this arch and 10 epochs - uping epochs
# 20 epochs - mem problem
#back to 10 epochs, changing first layer kernel from 15 to 25


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.BatchNorm1d(22),
            nn.Conv1d(22, 44, kernel_size=15, groups=22), #32x1000
            nn.BatchNorm1d(44), 
            nn.ELU(),
            nn.AvgPool1d(kernel_size=75, stride=15)) #32 x 500
        self.layer2 = nn.Sequential(
            nn.Conv1d(44, 30, kernel_size=3), #8x250
            nn.BatchNorm1d(30),
            nn.ELU()) #8x250
#         self.fc1 = nn.Sequential(
#             nn.Linear(15904, 5000), #100
#             nn.BatchNorm1d(5000),
#             nn.ReLU())
#         self.fc2 = nn.Sequential(
#             nn.Linear(5000, 1000), #100
#             nn.BatchNorm1d(1000),
#             nn.ReLU())
#         self.fc3 = nn.Sequential(
#             nn.Linear(1000, 500), #100
#             nn.BatchNorm1d(500),
#             nn.ReLU())
        self.fc4 = nn.Sequential(
            nn.Linear(1770, 10), #100
            nn.BatchNorm1d(10),
            nn.ReLU())
        self.fc5 = nn.Linear(1770, 4)
        
    #basic forward - go through two conv layers + fc layer
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
#         out = self.fc1(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
#         out = self.fc4(out)
        out = self.fc5(out)
        return out

cnn = CNN()

for i in cnn.parameters():
    print i.shape

In [ ]:
tensor_x_train = torch.Tensor(cnn_training_data_X)
tensor_y_train = torch.LongTensor(cnn_training_data_Y)

print tensor_x_train.shape
print tensor_y_train.shape

tensor_x_val = torch.Tensor(cnn_validation_data_X)
tensor_y_val = torch.LongTensor(cnn_validation_data_Y)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(tensor_x_train,tensor_y_train)
val_dataset = torch.utils.data.TensorDataset(tensor_x_val,tensor_y_val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [ ]:
print len(train_loader)
print len(val_loader)

In [ ]:
import torch
# torch.manual_seed(1337)
cnn = CNN()

# Loss and Optimizer

#Can also try nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate, weight_decay=0.03)

train_acc = []
val_acc = []
loss_history = []
          
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
    #     for i in range(cnn_training_data_X.shape[0]):
        images = Variable(images, requires_grad=True) #unsqueeze used to make a 4d tensor because 
    #     print images.shape
        labels = Variable(labels, volatile=True)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss_history.append(loss)
        loss.backward()
        optimizer.step()
        del outputs
        del images, labels
        gc.collect()

        if (i+1) % 20 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(cnn_training_data_X.shape)//batch_size, loss.data[0]))
            gc.collect()
            max_mem_used = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
            print("{:.2f} MB".format(max_mem_used /(1024*1024)))
 #           print ("mem usage",memory_usage())
        del loss
    
    images = Variable(torch.Tensor(cnn_training_data_X), volatile=True)
    labels = torch.LongTensor(cnn_training_data_Y)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0) #1800
#     print (predicted == labels).sum().int()
#     print type((predicted == labels).sum().int())
    correct = (predicted == labels).sum()
    del outputs
    del images, labels
    print(correct)
    print('Train Accuracy: %d %%' % (100.0 * int(correct) / float(total)))
    train_acc.append((100.0 * int(correct) / float(total)))

    cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    images = Variable(torch.Tensor(cnn_validation_data_X), volatile=True)
    labels = torch.LongTensor(cnn_validation_data_Y)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum()
    del outputs
    del images, labels
    print('validation Accuracy: %d %%' % (100.0 * int(correct) / float(total)))
    val_acc.append((100.0 * int(correct) / float(total)))

In [ ]:
# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
images = Variable(torch.Tensor(cnn_test_data_X))
labels = torch.LongTensor(cnn_test_data_Y)
outputs = cnn(images)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100.0 * int(correct) / float(total)))

In [ ]:
#plot train/val accuracies
print train_acc
plt.title("Train/Test Acc")
plt.xlabel("epoch")
plt.ylabel('acc')
plt.plot(train_acc, color='red')
plt.plot(val_acc, color='blue')
plt.show()

In [ ]:
from guppy import hpy; h=hpy()

In [ ]:
h.heap()

In [ ]:
h.iso(1,[],{})